## Nondeterministic Finite Automata
### Examples

_csc427, semester 212
<br>
university of miami
<br>
date: 3 february 2021
<br>
update: 7 february 2021_

<p>
(c) 2021 burton rosenberg
    <br>license below

### Introduction to nondeterminism

The question at hand is what can be computed given a particular machine model. This question has been reduced to set recognition problem, what subsets of some string set $\Sigma^*$ can be defined by and recognized by Finite Automata. One way of defining such sets is with Regular Expressions.

A Regular Expression (RE) a way of defining a subset of $\Sigma$ by declaring that any single letter of $\Sigma$ is an RE, the empty string and empty set are RE's, and then from this finitary basis, also sets defined by union, concatentation and (Kleene) star are RE's.

The point is, this <em>generates</em> sets which are exactly the sets that can be <em>recognized</em> by (deterministic) finite automata. That is, given an RE $\Omega$ we can define a FA $M$ such that for any $\sigma \in \Sigma^*$ generated $\Omega$ it is accepted by $M$, and if that $\sigma$ is not generated by $\Omega$ it is not accepted by $M$.

However, the situation is difficult. We have seen situations involving concatentation where the machine is not straightforward, as the machine must search among possibilities in its computation. A <em>Nondeterministic Finite Automata</em> captures this notion of computation, by proposing a machine that is free to branch, guess, or search possibilities, when exploring computation pathways.



### The NFA

We go right ahead to show code for an NFA (nondeterministic finite automata). Our implementation takes a position on nature of nondeterminism, which later on I declare to be the Schroedinger approach. We will trace alternative computations in a single computation trajectory.

The most important differences in the code for the nondeterministic FA compared to the deterministic FA are,

1. Current state is a set of states (new tokens can appear).
1. It is not an error if a key is missing. A missing key is is equivalent to a mapping for that key to the empty state set (a token can disappear).
3. The presence of epsilon moves. These are state transitions can on a special epsilon character, which can be inserted anywhere with any multiplicity in the input string.

The rules of computation are as follows.

1. An epsilon insertion to a string $\sigma \in \Sigma$ is any string $\hat{\sigma}\in \Sigma \cup \{\,\epsilon\,\}$ such that removing all $\epsilon$ from $\hat{\sigma}$ gives $\sigma$.
2. A run on a string is a list of states beginning with the start state and consistent with an epsilon insertion of the string and the transition function.
3. The NFA accepts if there is a run on the string that ends on an accepting state.

Here is a more detailed description of what computations are consistent with the transition function. 

Let the epsilon insertion for input be the string $w$ be written letter by letter as,

$$
w = w_1 w_2 \ldots w_n
$$

where $w_i$ could be a letter in $\Sigma$ or could be $\epsilon$, and let the computation $C$ be written as a sequence of states, 

$$
C = S_1 S_2 \ldots S_{n+1}.
$$

The computation is consistent if, 

1. $S_1$ is the start state.
2. For all $i$, state $S_{i+1}$ is among the possible allowed next states when in state $S_i$. I.e.:
$S_{i+1} \in\delta(S_i,w_i)$.



### Viewpoints on nondeterminism 

Three possible viewpoints:

1. Merlin model. Merlin gives Arthur the answer: an epsilon insertion of the string and a list of states. Arthur verifies. Merlin is always helpful and truthful. If there is an answer, Merlin will give it. You can either think of Arthur receving the full answers at first, or Merlin advising him at place there is a decision.
2. Hercules model. One by one Hercules tries all possible computations consistent with the all possible epsilon insertions until he he finds one that accepts, or has exhausted all possibilities. Hercules might arrange the computation as a tree of possibilities, and do either breath first or depth first search of the tree.
3. Schroedinger model: Multiple universes. We evolve a state set according to all possible worlds consistent with an epsilon insertion up to the n-th element of the string. This is the model we take, because it makes the NFA a DFA on the power set of states.




## NFA in Python


### Note! This code has a bug!

Fixing the bug is left for the problem set. Note there are two globals: verbose, for verbose operation, and allow_broken, which is there to remind us that this code has a bug, and to locate the bug.


In [1]:
verbose = True
allow_broken = True

class MachineModelNFA:
    """
    A machine description is a dictionary with,
        states: a list of states
        alphabet: a list of letters, and implicitly the empty letter ':'
        transitions: a dictionary with keys tuples (a state,a letter) to a set of states
        start: a state (the start state)
        accept: a list of states (the accepting states)
        
        the current_state is a set of states
        
    """
    
    def __init__(self,machine_description):
        self.states = machine_description['states']
        self.alphabet = machine_description['alphabet']
        self.transitions = machine_description['transitions']
        self.start_state = machine_description['start'] 
        self.accept_states = machine_description['accept']
        # enclosing in a list absorbs the ennumerate
        self.current_state = self.epsilon_close(set([self.start_state])) 
        # the : is reserved
        assert ':' not in self.alphabet
    
    def epsilon_one_step(self,state_set):
        e = set()
        for state in state_set:
            t = (state,':')
            if t in self.transitions:
                e = e.union(self.transitions[t])
        return e.union(state_set)
    
    def epsilon_close(self,state_set):
        assert(allow_broken)
        # this is broken because an epsilon closer follows 0 or more epsilon 
        # moves; here we only follow 0 or 1 epsilon moves.
        return self.epsilon_one_step(state_set)


    def do_transition(self,letter):
        new_current = set()
        for state in self.current_state:
            if (state,letter) in self.transitions:
                new_current = new_current.union(self.transitions[(state,letter)])
        self.current_state = self.epsilon_close(new_current)
    
    def compute(self,word):
        self.current_state = self.epsilon_close(set([self.start_state]))
        if verbose : print("initial state:", self.current_state)
        for w in word:
            self.do_transition(w)
            if verbose : print("char:",w,"state:",self.current_state)
        return len(self.current_state.intersection(self.accept_states))>0


class TestMachineNFA:
    """
    A class that constructs a MachineModelNFA from a Machine Description, 
    and can run a test. The test object is a list of pairs, 
        [(string,case), ... ]
    where case is True or False, if the string is, or is not, in the 
    language.
    """
    
    def __init__(self,machine_description):
        self.machine = MachineModelNFA(machine_description)
      
    def run(self,tests):
        print('running tests ...')
        for (t,r) in (tests):
            if self.machine.compute(t) != r:
                print(r,'\t|'+t+'|','\tWRONG, ABORT')
                return False
            print(r,'\t|'+t+'|','\tOK')
        return True


## Examples from the textbook

In [2]:
# exmples from ed 2 of Sipser

# N1 recognizes a 0,1 string containing 11 or 101

N1 = {
    'states':['Q1','Q2','Q3','Q4'],
    'alphabet':['0','1'],
    'transitions':{
        ('Q1','0'):['Q1'],('Q1','1'):['Q1','Q2'],
        ('Q2','0'):['Q3'],('Q2',':'):['Q3'],
        ('Q3','1'):['Q4'],
        ('Q4','0'):['Q4'],('Q4','1'):['Q4']
    },
    'start':'Q1',
    'accept':['Q4']
}

tests = [('101',True),('00011000',True),('10010010100',True),
         ('1001001001001',False),('00000',False),('',False)
        ]

tm = TestMachineNFA(N1)
tm.run(tests)




running tests ...
initial state: {'Q1'}
char: 1 state: {'Q1', 'Q2', 'Q3'}
char: 0 state: {'Q1', 'Q3'}
char: 1 state: {'Q4', 'Q2', 'Q3', 'Q1'}
True 	|101| 	OK
initial state: {'Q1'}
char: 0 state: {'Q1'}
char: 0 state: {'Q1'}
char: 0 state: {'Q1'}
char: 1 state: {'Q1', 'Q2', 'Q3'}
char: 1 state: {'Q4', 'Q2', 'Q3', 'Q1'}
char: 0 state: {'Q4', 'Q3', 'Q1'}
char: 0 state: {'Q4', 'Q1'}
char: 0 state: {'Q4', 'Q1'}
True 	|00011000| 	OK
initial state: {'Q1'}
char: 1 state: {'Q1', 'Q2', 'Q3'}
char: 0 state: {'Q1', 'Q3'}
char: 0 state: {'Q1'}
char: 1 state: {'Q1', 'Q2', 'Q3'}
char: 0 state: {'Q1', 'Q3'}
char: 0 state: {'Q1'}
char: 1 state: {'Q1', 'Q2', 'Q3'}
char: 0 state: {'Q1', 'Q3'}
char: 1 state: {'Q4', 'Q2', 'Q3', 'Q1'}
char: 0 state: {'Q4', 'Q3', 'Q1'}
char: 0 state: {'Q4', 'Q1'}
True 	|10010010100| 	OK
initial state: {'Q1'}
char: 1 state: {'Q1', 'Q2', 'Q3'}
char: 0 state: {'Q1', 'Q3'}
char: 0 state: {'Q1'}
char: 1 state: {'Q1', 'Q2', 'Q3'}
char: 0 state: {'Q1', 'Q3'}
char: 0 state: {'Q1'}
c

True

In [3]:
# N2 strings over 0,1 with a 1 third from the end

N2 = {
    'states':['Q1','Q2','Q3','Q4'],
    'alphabet':['0','1'],
    'transitions':{
        ('Q1','0'):['Q1'],('Q1','1'):['Q1','Q2'],
        ('Q2','0'):['Q3'],('Q2','1'):['Q3'],
        ('Q3','0'):['Q4'],('Q3','1'):['Q4']
    },
    'start':'Q1',
    'accept':['Q4']
}


tests = [('101',True),('00011100',True),('10010111',True),
         ('101001001',False),('00000',False),('',False)
        ]

tm = TestMachineNFA(N2)
tm.run(tests)



running tests ...
initial state: {'Q1'}
char: 1 state: {'Q1', 'Q2'}
char: 0 state: {'Q1', 'Q3'}
char: 1 state: {'Q4', 'Q1', 'Q2'}
True 	|101| 	OK
initial state: {'Q1'}
char: 0 state: {'Q1'}
char: 0 state: {'Q1'}
char: 0 state: {'Q1'}
char: 1 state: {'Q1', 'Q2'}
char: 1 state: {'Q1', 'Q2', 'Q3'}
char: 1 state: {'Q4', 'Q1', 'Q2', 'Q3'}
char: 0 state: {'Q4', 'Q1', 'Q3'}
char: 0 state: {'Q4', 'Q1'}
True 	|00011100| 	OK
initial state: {'Q1'}
char: 1 state: {'Q1', 'Q2'}
char: 0 state: {'Q1', 'Q3'}
char: 0 state: {'Q4', 'Q1'}
char: 1 state: {'Q1', 'Q2'}
char: 0 state: {'Q1', 'Q3'}
char: 1 state: {'Q4', 'Q1', 'Q2'}
char: 1 state: {'Q1', 'Q2', 'Q3'}
char: 1 state: {'Q4', 'Q1', 'Q2', 'Q3'}
True 	|10010111| 	OK
initial state: {'Q1'}
char: 1 state: {'Q1', 'Q2'}
char: 0 state: {'Q1', 'Q3'}
char: 1 state: {'Q4', 'Q1', 'Q2'}
char: 0 state: {'Q1', 'Q3'}
char: 0 state: {'Q4', 'Q1'}
char: 1 state: {'Q1', 'Q2'}
char: 0 state: {'Q1', 'Q3'}
char: 0 state: {'Q4', 'Q1'}
char: 1 state: {'Q1', 'Q2'}
False 	|10

True

In [4]:
# N3 accepts 0^i for i even or i divisible by 3

N3 = {
    'states':['S','ODD','EVEN','M0','M1','M2'],
    'alphabet':['0'],
    'transitions':{
        ('S',':'):['EVEN','M0'],
        ('EVEN','0'):['ODD'],('ODD','0'):['EVEN'],
        ('M0','0'):['M1'],('M1','0'):['M2'],
        ('M2','0'):['M0']
    },
    'start':'S',
    'accept':['EVEN','M0']
}


tests = [('',True),('0',False),('00',True),('000',True),
         ('0000',True),('00000',False),('000000',True),('0000000',False),
         ('00000000',True),('000000000',True),('0000000000',True),
         ('00000000000',False),('000000000000',True),
         ('0000000000000',False),('00000000000000',True),
        ]

tm = TestMachineNFA(N3)
tm.run(tests)


running tests ...
initial state: {'S', 'EVEN', 'M0'}
True 	|| 	OK
initial state: {'S', 'EVEN', 'M0'}
char: 0 state: {'ODD', 'M1'}
False 	|0| 	OK
initial state: {'S', 'EVEN', 'M0'}
char: 0 state: {'ODD', 'M1'}
char: 0 state: {'EVEN', 'M2'}
True 	|00| 	OK
initial state: {'S', 'EVEN', 'M0'}
char: 0 state: {'ODD', 'M1'}
char: 0 state: {'EVEN', 'M2'}
char: 0 state: {'ODD', 'M0'}
True 	|000| 	OK
initial state: {'S', 'EVEN', 'M0'}
char: 0 state: {'ODD', 'M1'}
char: 0 state: {'EVEN', 'M2'}
char: 0 state: {'ODD', 'M0'}
char: 0 state: {'EVEN', 'M1'}
True 	|0000| 	OK
initial state: {'S', 'EVEN', 'M0'}
char: 0 state: {'ODD', 'M1'}
char: 0 state: {'EVEN', 'M2'}
char: 0 state: {'ODD', 'M0'}
char: 0 state: {'EVEN', 'M1'}
char: 0 state: {'ODD', 'M2'}
False 	|00000| 	OK
initial state: {'S', 'EVEN', 'M0'}
char: 0 state: {'ODD', 'M1'}
char: 0 state: {'EVEN', 'M2'}
char: 0 state: {'ODD', 'M0'}
char: 0 state: {'EVEN', 'M1'}
char: 0 state: {'ODD', 'M2'}
char: 0 state: {'EVEN', 'M0'}
True 	|000000| 	OK
initi

True

In [5]:


# the finite language, { aa, bb }
# written as a union of two machines, an A machine and a B machine.

N_AA_BB = {
    'states':['S','A0','A1','A2','B0','B1','B2'],
    'alphabet': ['a','b'],
    'transitions': {
            ('S',':'):['A0','B0'],
            ('A0','a'):['A1'],
            ('A1','a'):['A2'],
            ('B0','b'):['B1'],
            ('B1','b'):['B2'],
            },
    'start': 'S',
    'accept': ['A2','B2']   
}

tests = [
    ('',False),
    ('aa',True),
    ('bb',True),
    ('a',False),
    ('bbb',False)
    ]

tm = TestMachineNFA(N_AA_BB)
tm.run(tests)

#{𝑤|𝑤 can be written as 𝑌𝑍 where 𝑌 has exactly two y's, and 𝑍 has exactly two z's}.
# written as the concatenation of two languages, with an epsilon move from state
# Y2 to Z0 which "guesses" where the string is divided into Y and Z parts.

N_YYZZ = {
    'states':['Y0','Y1','Y2','Z0','Z1','Z2'],
    'alphabet':['x','y','z'],
    'transitions':{
        ('Y0','x'):['Y0'],('Y0','z'):['Y0'],('Y0','y'):['Y1'],
        ('Y1','x'):['Y1'],('Y1','z'):['Y1'],('Y1','y'):['Y2'],
        ('Y2',':'):['Z0'],('Y2','x'):['Y2'],('Y2','z'):['Y2','Z1'],
        ('Z0','x'):['Z0'],('Z0','y'):['Z0'],('Z0','z'):['Z1'],
        ('Z1','x'):['Z1'],('Z1','y'):['Z1'],('Z1','z'):['Z2'],
        ('Z2','x'):['Z2'],('Z2','y'):['Z2']},
    'start':'Y0',
    'accept':['Z2']
}


tests =  [
    ('yyyzzz',False),
    ('yyzyzz',True),
    ('xyyxxyxzxzzxx',False),
    ('xyxyxxzyzxxxxzx',True),
    ('xyxyxxzyzxxzxzx',False),
    ('xyxyxxzzxxzyxzx',True),
]

tm = TestMachineNFA(N_YYZZ)
tm.run(tests)


running tests ...
initial state: {'B0', 'A0', 'S'}
False 	|| 	OK
initial state: {'B0', 'A0', 'S'}
char: a state: {'A1'}
char: a state: {'A2'}
True 	|aa| 	OK
initial state: {'B0', 'A0', 'S'}
char: b state: {'B1'}
char: b state: {'B2'}
True 	|bb| 	OK
initial state: {'B0', 'A0', 'S'}
char: a state: {'A1'}
False 	|a| 	OK
initial state: {'B0', 'A0', 'S'}
char: b state: {'B1'}
char: b state: {'B2'}
char: b state: set()
False 	|bbb| 	OK
running tests ...
initial state: {'Y0'}
char: y state: {'Y1'}
char: y state: {'Y2', 'Z0'}
char: y state: {'Z0'}
char: z state: {'Z1'}
char: z state: {'Z2'}
char: z state: set()
False 	|yyyzzz| 	OK
initial state: {'Y0'}
char: y state: {'Y1'}
char: y state: {'Y2', 'Z0'}
char: z state: {'Y2', 'Z1', 'Z0'}
char: y state: {'Z1', 'Z0'}
char: z state: {'Z2', 'Z1'}
char: z state: {'Z2'}
True 	|yyzyzz| 	OK
initial state: {'Y0'}
char: x state: {'Y0'}
char: y state: {'Y1'}
char: y state: {'Y2', 'Z0'}
char: x state: {'Y2', 'Z0'}
char: x state: {'Y2', 'Z0'}
char: y state: {

True

------

<a rel="license" href="http://creativecommons.org/licenses/by/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by/4.0/88x31.png" /></a><br />This work is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by/4.0/">Creative Commons Attribution 4.0 International License</a>.

------